<a href="https://colab.research.google.com/github/jamestheengineer/data-science-from-scratch-Python/blob/master/Chapter_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clustering chapter. Example of unsupervised learning.

# Only do this once per VM, otherwise you'll get multiple clones and nested directories
!git clone https://github.com/jamestheengineer/data-science-from-scratch-Python.git
%cd data-science-from-scratch-Python/
!pip install import-ipynb
!pip install pillow
import import_ipynb

Cloning into 'data-science-from-scratch-Python'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 489 (delta 66), reused 0 (delta 0), pack-reused 387
Receiving objects: 100% (489/489), 1.18 MiB | 2.92 MiB/s, done.
Resolving deltas: 100% (306/306), done.
/content/data-science-from-scratch-Python
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=7ed9f717796cf030fdd655ec1249f6b30856ec684dda250b1520a19554d6eaa4
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [ ]:
from Chapter_04 import Vector

def num_differences(v1: Vector, v2: Vector) -> int:
  assert len(v1) == len(v2)
  return len([x1 for x1, x2 in zip(v1,v2) if x1 != x2])

assert num_differences([1,2,3],[2,1,3]) == 2
assert num_differences([1,2],[1,2]) == 0

In [ ]:
from typing import List
from Chapter_04 import vector_mean

def cluster_means(k: int,
                  inputs: List[Vector],
                  assignments: List[int]) -> List[Vector]:
  # cluster[i] contains the inputs whose assignment is i
  clusters = [[] for i in range(k)]
  for input, assignment in zip(inputs, assignments):
    clusters[assignment].append(input)

  # if a cluster is empty, just use a random point
  return [vector_mean(cluster) if cluster else random.choice(inputs)
          for cluster in clusters]

In [ ]:
import itertools
import random
import tqdm
from Chapter_04 import squared_distance

class KMeans:
  def __init__(self, k: int) -> None:
    self.k = k
    self.means = None

  def classify(self, input: Vector) -> int:
    """return the index of the cluster closes to the input"""
    return min(range(self.k),
               key=lambda i: squared_distance(input, self.means[i]))
    
  def train(self, inputs: List[Vector]) -> None:
    # Start with random assignments
    assignments = [random.randrange(self.k) for _ in inputs]

    with tqdm.tqdm(itertools.count()) as t:
      for _ in t:
        # Compute means and find new assignments
        self.means = cluster_means(self.k, inputs, assignments)
        new_assignments = [self.classify(input) for input in inputs]

        # Check how many assignments changed and if we're done
        num_changed = num_differences(assignments, new_assignments)
        if num_changed == 0:
          return
        
        # Otherwise keep the new assignments, and compute new means
        assignments = new_assignments
        self.means = cluster_means(self.k, inputs, assignments)
        t.set_description(f"changed: {num_changed} / {len(inputs)}")

In [ ]:
# Meetup location example
inputs: List[List[float]] = [[-14,-5],[13,13],[20,23],[-19,-11],[-9,-16],[21,27],[-49,15],[26,13],[-46,5],[-34,-1],[11,15],[-49,0],[-22,-16],[19,28],[-12,-8],[-13,-19],[-41,8],[-11,-6],[-25,-9],[-18,-3]]

random.seed(12)
clusterer = KMeans(k=3)
clusterer.train(inputs)
means = sorted(clusterer.means)

assert len(means) == 3

# Check that the means are close to what we expect
assert squared_distance(means[0], [-44, 5]) < 1
assert squared_distance(means[1], [-16, -10]) < 1
assert squared_distance(means[2], [18, 20]) < 1 
print(means)

In [ ]:
# Only cash for two meetups, so...
random.seed(0)
clusterer = KMeans(k=2)
clusterer.train(inputs)
means = sorted(clusterer.means)

assert len(means) == 2
assert squared_distance(means[0], [-26, -5]) < 1
assert squared_distance(means[1], [18, 20]) < 1

In [ ]:
# Choosing k by looking at the "bend"
from matplotlib import pyplot as plt 

def squared_clustering_errors(inputs: List[Vector], k: int) -> float:
  """finds the total squared error from k-means clustering the inputs"""
  clusterer = KMeans(k)
  clusterer.train(inputs)
  means = clusterer.means
  assignments = [clusterer.classify(input) for input in inputs]

  return sum(squared_distance(input, means[cluster])
              for input, cluster in zip(inputs, assignments))

# now plot from 1 up to len(inputs) clusters

ks = range(1, len(inputs) + 1)
errors = [squared_clustering_errors(inputs, k) for k in ks]

plt.plot(ks, errors)
plt.xticks(ks)
plt.xlabel("k")
plt.ylabel("total squared error")
plt.title("Total Error vs. # of Clusters")
plt.show()